# LIBRAIRIES

In [1]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import time
import seaborn as sn
from sklearn import *
from sklearn.linear_model import *
from sklearn.neural_network import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import ADASYN
from sklearn.decomposition import PCA
import pickle

### CONSTANTES :

In [2]:
#entrer le path de votre dataset
path_name="/Users/edouardnadaud/Desktop/DATACAMP_PYTHON_2/Dataset/trainset/trainset.csv"
dataPi = pd.read_csv(path_name)

In [3]:
X_dataPi = dataPi[dataPi.columns[1:-1]]
y_dataPi = dataPi[dataPi.columns[-1]]

In [4]:
def dist(point1, point2, df):
    point1, point2 = str(point1), str(point2)
    return  (  (df[' x_' + point2] - df[' x_' + point1])**(2) +  (df[' y_' + point2] - df[' y_' + point1])**(2)   )**(0.5)

def dist_normalisee(point1, point2, df, coef): # normalisée par la distance interoculaire
    point1, point2 = str(point1), str(point2)
    return  (((  (df[' x_' + point2] - df[' x_' + point1])**(2) +  (df[' y_' + point2] - df[' y_' + point1])**(2)   )**(0.5))/coef)

def angl(point1, point2, point3, df):
    point1, point2, point3 = str(point1), str(point2), str(point3)
    a = np.array([df[' x_' + point1],df[' y_' + point1]])
    b = np.array([df[' x_' + point2],df[' y_' + point2]])
    c = np.array([df[' x_' + point3],df[' y_' + point3]])
    
    angle = np.degrees(np.arctan(c[1]-b[1], c[0]-b[0]) - np.arctan(a[1]-b[1], a[0]-b[0]))
 
    angle[angle<0] = angle[angle<0] + 360
    return angle



In [9]:
X_new_ft = X_dataPi.copy()

X_new_ft['distOculaire'] = (  (X_new_ft.loc[: , " x_42":" x_47"].mean(axis=1) - X_new_ft.loc[: , " x_36":" x_41"].mean(axis=1))**(2) +  (X_new_ft.loc[: , " y_42":" y_47"].mean(axis=1) - X_new_ft.loc[: , " y_36":" y_41"].mean(axis=1))**(2)   )**(0.5)


for i in range(0,68):
    for j in range(i,68): # afin de ne pas recalculer deux fois la meme distance
        if (i != j): # on ne calcule pas la distance d'un point avec lui même
            #X_new_ft['dist'+str(i)+"_"+str(j)]  = dist(i, j, X_dataPi)
            X_new_ft['dist_norm'+str(i)+"_"+str(j)]  = dist_normalisee(i, j, X_dataPi,X_new_ft['distOculaire'])

for i in range(0,68):
    for j in range(0,68): # afin de ne pas recalculer deux fois le même angle
        for k in range(i,68):
            if (i != j and j != k and i != k): # on ne calcule pas d'angle entre 3 points quand deux d'entre eux sont les mêmes             
                X_new_ft['angl'+str(i)+"_"+str(j)+"_"+str(k)] = angl(i, j, k, X_dataPi)
    print('clacul pour les angles a=',i)

X_new_ft.drop(X_new_ft.iloc[:, 0:137], inplace = True, axis = 1) 

ColonnesDf = X_new_ft.columns
indexDf = X_new_ft.index
standardScaler = StandardScaler()
X_new_ft = standardScaler.fit_transform(X_new_ft)

X_new_ft = pd.DataFrame(X_new_ft, index=indexDf, columns=ColonnesDf)

clacul pour les angles a= 0
clacul pour les angles a= 1
clacul pour les angles a= 2


#### PCA

In [6]:
X_new_ft_PCA = X_new_ft.copy()
pca = PCA(n_components=dataPi.shape[0])
pca.fit(X_new_ft_PCA)
XPCA = pca.transform(X_new_ft_PCA)

In [7]:
XPCA=pd.DataFrame(data=XPCA)

XPCA.to_csv('/Users/edouardnadaud/Desktop/DATACAMP_PYTHON_2/Projet_final/python_rendu/Dataframe_with_feature_extraction.csv')


# save the classifier
with open('/Users/edouardnadaud/Desktop/DATACAMP_PYTHON_2/Projet_final/python_rendu/PCA.pkl', 'wb') as fid:
    pickle.dump(pca, fid) 